In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import re
import time
from datetime import datetime, timedelta
from pandas.tseries.offsets import *
import locale
locale.setlocale(locale.LC_TIME, "de_DE.UTF-8")

%matplotlib inline

In [2]:
df = pd.read_csv('speedtest.txt')

In [3]:
df.head()

,So 29. Jan 18:00:01 CET 2017 Retrieving speedtest.net configuration...
0,Retrieving speedtest.net server list...
1,Testing from Vodafone (31.19.116.65)...
2,Selecting best server based on latency...
3,Hosted by wilhelm.tel (Norderstedt) [16.76 km]...
4,Testing download speed...........................


In [4]:
with open('speedtest.txt.alt', 'r') as myfile:
    speed1=myfile.read()

In [5]:
with open('speedtest.txt', 'r') as myfile:
    speed2=myfile.read()

In [6]:
speed = speed1 + speed2

In [7]:
speed = speed.split('/n\n')

In [8]:
speed[0:2]

['Fr 4. Dez 06:24:55 CET 2015 Retrieving speedtest.net configuration...\nRetrieving speedtest.net server list...\nTesting from Vodafone Kabel Deutschland (31.17.185.251)...\nSelecting best server based on latency...\nHosted by wilhelm.tel (Norderstedt) [14.59 km]: 31.589 ms\nTesting download speed........................................\nDownload: 54.13 Mbit/s\nTesting upload speed..................................................\nUpload: 5.33 Mbit/s ',
 'Fr 4. Dez 21:30:01 CET 2015 Retrieving speedtest.net configuration...\nRetrieving speedtest.net server list...\nTesting from Vodafone Kabel Deutschland (31.17.185.251)...\nSelecting best server based on latency...\nHosted by C&S Breitband GmbH (Bad Oldesloe) [37.30 km]: 31.824 ms\nTesting download speed........................................\nDownload: 49.51 Mbit/s\nTesting upload speed..................................................\nUpload: 5.33 Mbit/s ']

In [9]:
list_of_dicts = []
fail_list = []
for test in speed:
    try:
        tempdict={}
        tempdict['date'] = re.findall(r'[A-Z][a-z] \d+. \w+', test)[0]+' '+re.findall(r'\d+:\d+:\d+', test)[0]
        # Do not want to have my ip address on the internet sorry
        #tempdict['my_ip'] = re.findall(r'\d+.\d+.\d+.\d+', test)[0]
        tempdict['test_server'] = re.findall(r'Hosted by (.*?)\(', test)[0].strip()
        tempdict['test_server_location'] = re.findall(r'Hosted by .*?\((.*?)\)', test)[0]
        tempdict['test_server_distance'] = re.findall(r'\[(.*?)km\]', test)[0]
        tempdict['test_server_latency'] = re.findall(r'(\d+.\d+) ms', test)[0]
        tempdict['download'] = re.findall(r'Download: (\d+.\d+)', test)[0]
        tempdict['upload'] = re.findall(r'Upload: (\d+.\d+)', test)[0]
        list_of_dicts.append(tempdict)
    except:
        pass
        fail_list.append(test)

In [10]:
df = pd.DataFrame(list_of_dicts)
df

,date,download,test_server,test_server_distance,test_server_latency,test_server_location,upload
0,Fr 4. Dez 06:24:55,54.13,wilhelm.tel,14.59,31.589,Norderstedt,5.33
1,Fr 4. Dez 21:30:01,49.51,C&S Breitband GmbH,37.30,31.824,Bad Oldesloe,5.33
2,Fr 4. Dez 21:40:02,51.08,C&S Breitband GmbH,37.30,30.336,Bad Oldesloe,5.44
3,Fr 4. Dez 21:50:02,49.49,wilhelm.tel,14.59,31.136,Norderstedt,5.32
4,Fr 4. Dez 22:00:01,49.71,C&S Breitband GmbH,37.30,30.492,Bad Oldesloe,5.49
5,Fr 4. Dez 22:10:01,49.78,wilhelm.tel,14.59,31.129,Norderstedt,5.21
6,Fr 4. Dez 22:20:02,51.01,wilhelm.tel,14.59,29.163,Norderstedt,5.38
7,Fr 4. Dez 22:30:01,51.18,Stadtwerke Neumünster GmbH,54.95,29.095,Neumuenster,5.42
8,Sa 5. Dez 09:10:02,55.60,C&S Breitband GmbH,37.30,31.873,Bad Oldesloe,5.38
9,Sa 5. Dez 09:20:02,54.29,wilhelm.tel,14.59,32.742,Norderstedt,5.30


In [11]:
df.dtypes

date                    object
download                object
test_server             object
test_server_distance    object
test_server_latency     object
test_server_location    object
upload                  object
dtype: object

In [12]:
df.date = pd.to_datetime(df.date, format = '%a %d. %b %H:%M:%S') + pd.DateOffset(years=117)

In [13]:
df.loc[:652].date = df.loc[:652].date - pd.DateOffset(years=1)
df.loc[11990:].date = df.loc[11990:].date + pd.DateOffset(years=1)

/home/jls/.local/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [14]:
df

,date,download,test_server,test_server_distance,test_server_latency,test_server_location,upload
0,2016-12-04 06:24:55,54.13,wilhelm.tel,14.59,31.589,Norderstedt,5.33
1,2016-12-04 21:30:01,49.51,C&S Breitband GmbH,37.30,31.824,Bad Oldesloe,5.33
2,2016-12-04 21:40:02,51.08,C&S Breitband GmbH,37.30,30.336,Bad Oldesloe,5.44
3,2016-12-04 21:50:02,49.49,wilhelm.tel,14.59,31.136,Norderstedt,5.32
4,2016-12-04 22:00:01,49.71,C&S Breitband GmbH,37.30,30.492,Bad Oldesloe,5.49
5,2016-12-04 22:10:01,49.78,wilhelm.tel,14.59,31.129,Norderstedt,5.21
6,2016-12-04 22:20:02,51.01,wilhelm.tel,14.59,29.163,Norderstedt,5.38
7,2016-12-04 22:30:01,51.18,Stadtwerke Neumünster GmbH,54.95,29.095,Neumuenster,5.42
8,2016-12-05 09:10:02,55.60,C&S Breitband GmbH,37.30,31.873,Bad Oldesloe,5.38
9,2016-12-05 09:20:02,54.29,wilhelm.tel,14.59,32.742,Norderstedt,5.30


In [15]:
df.loc[652:653].date

652   2016-12-31 23:00:02
653   2017-01-01 00:00:01
Name: date, dtype: datetime64[ns]

In [16]:
df.loc[11989:11990].date

11989   2017-12-31 22:00:01
11990   2018-01-01 02:00:01
Name: date, dtype: datetime64[ns]

In [17]:
df.download = df.download.astype(float)
df.upload = df.upload.astype(float)
df.test_server_distance = df.test_server_distance.astype(float)
df.test_server_latency = df.test_server_latency.astype(float)

In [18]:
df.dtypes

date                    datetime64[ns]
download                       float64
test_server                     object
test_server_distance           float64
test_server_latency            float64
test_server_location            object
upload                         float64
dtype: object

In [19]:
df.download.min()

0.0

In [20]:
df[df.download < 10]

,date,download,test_server,test_server_distance,test_server_latency,test_server_location,upload
545,2016-12-27 09:00:01,0.00,Studio Funk GmbH & Co. KG,7.50,31.172,Hamburg,3.22
593,2016-12-29 09:00:01,0.00,Studio Funk GmbH & Co. KG,8.27,28.636,Hamburg,0.00
1153,2017-01-21 21:00:01,7.06,Stadtwerke Neumünster GmbH,53.86,69.172,Neumunster,0.36
1414,2017-02-01 18:00:01,5.97,Studio Funk GmbH & Co. KG,1.90,30.755,Hamburg,0.59
1847,2017-02-19 23:00:01,8.16,Buchholz Digital GmbH,25.99,28.548,Buchholz in der Nordheide,4.06
1919,2017-02-22 23:00:01,8.71,Buchholz Digital GmbH,26.44,28.336,Buchholz in der Nordheide,5.33
1922,2017-02-23 02:00:01,8.95,Buchholz Digital GmbH,26.44,28.874,Buchholz in der Nordheide,5.50
1930,2017-02-23 11:00:01,8.39,Buchholz Digital GmbH,26.44,27.522,Buchholz in der Nordheide,4.84
1960,2017-02-24 17:00:01,7.69,Buchholz Digital GmbH,26.44,28.488,Buchholz in der Nordheide,1.03
2002,2017-03-06 03:00:01,9.96,Buchholz Digital GmbH,26.45,29.307,Buchholz in der Nordheide,2.03


In [21]:
df.download.max()

57.72

In [22]:
df[df.download > 57]

,date,download,test_server,test_server_distance,test_server_latency,test_server_location,upload
4626,2017-01-30 17:00:01,57.30,C&S Breitband GmbH,36.86,29.788,Bad Oldesloe,5.45
4632,2017-01-30 23:00:02,57.37,C&S Breitband GmbH,36.80,31.155,Bad Oldesloe,5.48
4637,2017-01-31 04:00:01,57.32,C&S Breitband GmbH,36.80,28.722,Bad Oldesloe,5.46
4639,2017-01-31 06:00:02,57.12,C&S Breitband GmbH,36.80,32.016,Bad Oldesloe,5.47
4640,2017-01-31 07:00:01,57.38,C&S Breitband GmbH,36.80,32.869,Bad Oldesloe,5.35
4642,2017-01-31 09:00:01,57.47,C&S Breitband GmbH,36.80,29.301,Bad Oldesloe,5.50
4652,2017-01-31 19:00:01,57.72,C&S Breitband GmbH,36.80,33.030,Bad Oldesloe,5.51
4657,2017-02-01 00:00:02,57.54,C&S Breitband GmbH,36.80,33.663,Bad Oldesloe,5.53
4658,2017-02-01 01:00:01,57.41,C&S Breitband GmbH,36.80,30.521,Bad Oldesloe,5.55
4666,2017-02-01 09:00:02,57.08,C&S Breitband GmbH,36.80,32.800,Bad Oldesloe,5.37


In [23]:
df.describe()

,download,test_server_distance,test_server_latency,upload
count,14001.000000,14001.000000,14001.000000,14001.000000
mean,47.629351,20.770066,32.249527,5.239476
std,10.823985,14.245180,5.165900,0.627571
min,0.000000,0.340000,25.814000,0.000000
25%,45.970000,13.510000,29.940000,5.310000
50%,52.230000,16.550000,31.063000,5.380000
75%,54.350000,36.780000,32.573000,5.440000
max,57.720000,83.910000,199.277000,5.580000


In [24]:
df.test_server_location.value_counts()

Norderstedt                  6205
Bad Oldesloe                 3233
Hamburg                      3005
Buchholz in der Nordheide    1202
Neumunster                    233
Neumuenster                    75
Kiel                           44
Itzehoe                         3
Dusseldorf                      1
Name: test_server_location, dtype: int64

In [25]:
df.test_server.value_counts()

wilhelm.tel                     6205
C&S Breitband GmbH              2921
LWLcom GmbH                     1990
Buchholz Digital GmbH           1202
Studio Funk GmbH & Co. KG       1016
Vereinigte Stadtwerke GmbH       312
Stadtwerke Neumünster GmbH       308
TNG Stadtnetz GmbH                43
ITLogic EU                         3
ADDIX Internet Services GmbH       1
Name: test_server, dtype: int64

In [26]:
#df.my_ip.value_counts()

In [27]:
df.to_csv('speedtest_sorted.csv', index=False)

In [28]:
# charts in new notebook I guess